In [0]:
use mgiglia.dev_matthew_giglia_price_transparency;

In [0]:
select * from allowed_amount_json_bronze;

In [0]:
DROP TABLE IF EXISTS mgiglia.pt_allowed_amounts.file_dim;

In [0]:
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.file_dim (
  file_dim_sk STRING NOT NULL
  ,index_file_source_id STRING
  ,file_path STRING
  ,file_name STRING
  ,file_size BIGINT
  ,file_block_start BIGINT
  ,file_block_length BIGINT
  ,file_modification_time TIMESTAMP
  ,ingest_time TIMESTAMP
  ,rcrd_timestamp TIMESTAMP
  ,last_updated_on TIMESTAMP
  ,version STRING
  ,CONSTRAINT dim_files_pk PRIMARY KEY (file_dim_sk)
)
COMMENT "File dimension containing metadata about source files"
AS SELECT DISTINCT
  index_file_source_id as file_dim_sk
  ,index_file_source_id
  ,file_metadata.file_path AS file_path
  ,file_metadata.file_name AS file_name
  ,file_metadata.file_size AS file_size
  ,file_metadata.file_block_start AS file_block_start
  ,file_metadata.file_block_length AS file_block_length
  ,file_metadata.file_modification_time AS file_modification_time
  ,ingest_time
  ,rcrd_timestamp
  ,last_updated_on::timestamp
  ,version
FROM STREAM(allowed_amount_json_bronze);

In [0]:
select * from mgiglia.pt_allowed_amounts.file_dim limit 100;

In [0]:
DROP TABLE IF EXISTS mgiglia.pt_allowed_amounts.plan_dim;

In [0]:
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.plan_dim (
  plan_dim_sk STRING NOT NULL,
  plan_name STRING,
  plan_id STRING,
  plan_id_type STRING,
  plan_market_type STRING,
  reporting_entity_name STRING,
  reporting_entity_type STRING,
  CONSTRAINT dim_plans_pk PRIMARY KEY (plan_dim_sk)
)
COMMENT "Insurance plans dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(COALESCE(plan_name, ''), '|', COALESCE(plan_id, ''), '|', COALESCE(plan_id_type, ''), '|', COALESCE(plan_market_type, ''), '|', COALESCE(reporting_entity_name, ''), '|', COALESCE(reporting_entity_type, ''), '|', COALESCE(version, '')), 256) as plan_dim_sk
  ,plan_name
  ,plan_id
  ,plan_id_type
  ,plan_market_type
  ,reporting_entity_name
  ,reporting_entity_type
FROM STREAM(allowed_amount_json_bronze);

In [0]:
select * from mgiglia.pt_allowed_amounts.plan_dim limit 100;

In [0]:
SELECT DISTINCT
  t.*
  ,oon.col.
FROM 
  allowed_amount_json_bronze as t
  ,LATERAL EXPLODE(out_of_network) as oon

In [0]:
drop table if exists mgiglia.pt_allowed_amounts.billing_code_dim;

In [0]:
-- Billing Codes Dimension
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.billing_code_dim (
  billing_code_dim_sk STRING NOT NULL,
  billing_code STRING,
  billing_code_type STRING,
  billing_code_type_version STRING,
  name STRING,
  description STRING,
  CONSTRAINT dim_billing_codes_pk PRIMARY KEY (billing_code_dim_sk)
)
COMMENT "Medical billing codes dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(
    COALESCE(oon.col.billing_code, ''), '|',
    COALESCE(oon.col.billing_code_type, ''), '|',
    COALESCE(oon.col.billing_code_type_version, ''), '|',
    COALESCE(oon.col.name, ''), '|',
    COALESCE(oon.col.description, '')
  ), 256) as billing_code_dim_sk,
  oon.col.billing_code as billing_code,
  oon.col.billing_code_type as billing_code_type,
  oon.col.billing_code_type_version as billing_code_type_version,
  oon.col.name as name,
  oon.col.description as description
FROM STREAM(allowed_amount_json_bronze)
,LATERAL EXPLODE(out_of_network) as oon;

In [0]:
select * from mgiglia.pt_allowed_amounts.billing_code_dim limit 100

In [0]:
drop table if exists mgiglia.pt_allowed_amounts.tin_dim;

In [0]:
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.tin_dim (
  tin_dim_sk STRING NOT NULL,
  tin_type STRING,
  tin_value STRING,
  CONSTRAINT dim_tins_pk PRIMARY KEY (tin_dim_sk)
)
COMMENT "Tax Identification Numbers dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(COALESCE(tin_info.col.type, ''), '|', COALESCE(tin_info.col.value, '')), 256) as tin_dim_sk,
  tin_info.col.type as tin_type,
  tin_info.col.value as tin_value
FROM STREAM(allowed_amount_json_bronze)
,LATERAL EXPLODE(out_of_network) as oon
,LATERAL EXPLODE(oon.col.allowed_amounts) as allowed_amt
,LATERAL EXPLODE(array(allowed_amt.col.tin)) as tin_info;

In [0]:
select * from mgiglia.pt_allowed_amounts.tin_dim limit 100;

In [0]:
drop table if exists mgiglia.pt_allowed_amounts.provider_dim;

In [0]:
-- Providers Dimension
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.provider_dim (
  provider_dim_sk STRING NOT NULL,
  npi STRING,
  CONSTRAINT dim_providers_pk PRIMARY KEY (provider_dim_sk)
)
COMMENT "Healthcare providers dimension"
AS SELECT DISTINCT
  SHA2(CAST(FORMAT_NUMBER(CAST(npi_value.col AS DECIMAL(38,0)), '0') AS STRING), 256) as provider_dim_sk,
  CAST(FORMAT_NUMBER(CAST(npi_value.col AS DECIMAL(38,0)), '0') AS STRING) as npi
FROM STREAM(allowed_amount_json_bronze)
,LATERAL EXPLODE(out_of_network) as oon
,LATERAL EXPLODE(oon.col.allowed_amounts) as allowed_amt
,LATERAL EXPLODE(allowed_amt.col.payments) as payment
,LATERAL EXPLODE(payment.col.providers) as provider
,LATERAL EXPLODE(provider.col.npi) as npi_value;

In [0]:
select * from mgiglia.pt_allowed_amounts.provider_dim limit 100;

In [0]:
drop table if exists mgiglia.pt_allowed_amounts.service_code_dim;

In [0]:
-- BRIDGE TABLES

-- Service Codes Bridge Table
CREATE OR REFRESH STREAMING TABLE mgiglia.pt_allowed_amounts.service_code_dim (
  service_code_dim_sk STRING NOT NULL,
  billing_code_dim_sk STRING NOT NULL,
  service_code STRING,
  CONSTRAINT dim_service_codes_pk PRIMARY KEY (service_code_dim_sk),
  CONSTRAINT dim_service_codes_billing_code_fk FOREIGN KEY (billing_code_dim_sk) REFERENCES mgiglia.pt_allowed_amounts.billing_code_dim(billing_code_dim_sk)
)
COMMENT "Bridge table linking billing codes to service codes"
AS SELECT DISTINCT
  SHA2(CONCAT(billing_code_dim_sk, '|', service_code_value.col), 256) as service_code_dim_sk,
  billing_code_dim_sk,
  service_code_value.col as service_code
FROM (
  SELECT 
    SHA2(CONCAT(
    COALESCE(oon.col.billing_code, ''), '|',
    COALESCE(oon.col.billing_code_type, ''), '|',
    COALESCE(oon.col.billing_code_type_version, ''), '|',
    COALESCE(oon.col.name, ''), '|',
    COALESCE(oon.col.description, '')
  ), 256) as billing_code_dim_sk,
    service_code_element.col as service_code_value
  FROM STREAM(allowed_amount_json_bronze)
  ,LATERAL EXPLODE(out_of_network) as oon
  ,LATERAL EXPLODE(array(oon.col)) as oon_col
  ,LATERAL EXPLODE(oon_col) as service_code_value
  WHERE service_code_value.col IS NOT NULL
);

In [0]:
select * from mgiglia.pt_allowed_amounts.service_code_dim;

In [0]:
-- healthcare_pricing_pipeline.sql

-- DIMENSION TABLES

-- Files Dimension


-- Plans Dimension  
CREATE OR REFRESH STREAMING TABLE dim_plans (
  plan_id STRING NOT NULL,
  plan_name STRING,
  plan_id_type STRING,
  plan_market_type STRING,
  reporting_entity_name STRING,
  reporting_entity_type STRING,
  version STRING,
  CONSTRAINT dim_plans_pk PRIMARY KEY (plan_id)
)
COMMENT "Insurance plans dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(COALESCE(plan_name, ''), '|', COALESCE(reporting_entity_name, '')), 256) as plan_id,
  plan_name,
  plan_id_type,
  plan_market_type,
  reporting_entity_name,
  reporting_entity_type,
  version
FROM STREAM(source_healthcare_data);

-- Billing Codes Dimension
CREATE OR REFRESH STREAMING TABLE dim_billing_codes (
  billing_code_id STRING NOT NULL,
  billing_code STRING,
  billing_code_type STRING,
  billing_code_type_version STRING,
  name STRING,
  description STRING,
  billing_class STRING,
  CONSTRAINT dim_billing_codes_pk PRIMARY KEY (billing_code_id)
)
COMMENT "Medical billing codes dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(COALESCE(oon_element.billing_code, ''), '|', COALESCE(oon_element.billing_code_type, '')), 256) as billing_code_id,
  oon_element.billing_code,
  oon_element.billing_code_type,
  oon_element.billing_code_type_version,
  oon_element.name,
  oon_element.description,
  oon_element.billing_class
FROM STREAM(source_healthcare_data)
LATERAL VIEW EXPLODE(out_of_network) as oon
LATERAL VIEW EXPLODE(array(oon.element)) as oon_element;

-- TINs Dimension
CREATE OR REFRESH STREAMING TABLE dim_tins (
  tin_id STRING NOT NULL,
  tin_type STRING,
  tin_value STRING,
  CONSTRAINT dim_tins_pk PRIMARY KEY (tin_id)
)
COMMENT "Tax Identification Numbers dimension"
AS SELECT DISTINCT
  SHA2(CONCAT(COALESCE(tin_info.type, ''), '|', COALESCE(tin_info.value, '')), 256) as tin_id,
  tin_info.type as tin_type,
  tin_info.value as tin_value
FROM STREAM(source_healthcare_data)
LATERAL VIEW EXPLODE(out_of_network) as oon
LATERAL VIEW EXPLODE(oon.element.allowed_amounts) as allowed_amt
LATERAL VIEW EXPLODE(array(allowed_amt.element.tin)) as tin_info;

-- Providers Dimension
CREATE OR REFRESH STREAMING TABLE dim_providers (
  provider_id STRING NOT NULL,
  npi STRING,
  provider_name STRING,
  CONSTRAINT dim_providers_pk PRIMARY KEY (provider_id)
)
COMMENT "Healthcare providers dimension"
AS SELECT DISTINCT
  SHA2(CAST(npi_value AS STRING), 256) as provider_id,
  CAST(npi_value AS STRING) as npi,
  NULL as provider_name -- Not available in source data
FROM STREAM(source_healthcare_data)
LATERAL VIEW EXPLODE(out_of_network) as oon
LATERAL VIEW EXPLODE(oon.element.allowed_amounts) as allowed_amt
LATERAL VIEW EXPLODE(allowed_amt.element.payments) as payment
LATERAL VIEW EXPLODE(payment.element.providers) as provider
LATERAL VIEW EXPLODE(provider.element.npi) as npi_value;

-- BRIDGE TABLES

-- Service Codes Bridge Table
CREATE OR REFRESH STREAMING TABLE dim_service_codes (
  service_code_id STRING NOT NULL,
  billing_code_id STRING NOT NULL,
  service_code STRING,
  CONSTRAINT dim_service_codes_pk PRIMARY KEY (service_code_id)
)
COMMENT "Bridge table linking billing codes to service codes"
AS SELECT DISTINCT
  SHA2(CONCAT(billing_code_id, '|', service_code_value), 256) as service_code_id,
  billing_code_id,
  service_code_value as service_code
FROM (
  SELECT 
    SHA2(CONCAT(COALESCE(oon_element.billing_code, ''), '|', COALESCE(oon_element.billing_code_type, '')), 256) as billing_code_id,
    service_code_element as service_code_value
  FROM STREAM(source_healthcare_data)
  LATERAL VIEW EXPLODE(out_of_network) as oon
  LATERAL VIEW EXPLODE(array(oon.element)) as oon_element
  LATERAL VIEW EXPLODE(oon_element.service_code) as service_code_element
  WHERE service_code_element IS NOT NULL
);

-- FACT TABLE
CREATE OR REFRESH STREAMING TABLE fact_allowed_amounts (
  allowed_amount_id STRING NOT NULL,
  file_id STRING NOT NULL,
  plan_id STRING NOT NULL,
  provider_id STRING,
  billing_code_id STRING NOT NULL,
  tin_id STRING,
  allowed_amount DOUBLE,
  billed_charge DOUBLE,
  rcrd_timestamp TIMESTAMP,
  ingest_time TIMESTAMP,
  CONSTRAINT fact_allowed_amounts_pk PRIMARY KEY (allowed_amount_id)
)
COMMENT "Fact table containing allowed amounts for healthcare services"
AS SELECT 
  SHA2(CONCAT(
    file_id, '|', plan_id, '|', 
    COALESCE(provider_id, 'NULL'), '|', 
    billing_code_id, '|', 
    COALESCE(tin_id, 'NULL')
  ), 256) as allowed_amount_id,
  file_id,
  plan_id,
  provider_id,
  billing_code_id,
  tin_id,
  payment_element.allowed_amount,
  provider_element.billed_charge,
  rcrd_timestamp,
  ingest_time
FROM (
  SELECT 
    SHA2(index_file_source_id, 256) as file_id,
    SHA2(CONCAT(COALESCE(plan_name, ''), '|', COALESCE(reporting_entity_name, '')), 256) as plan_id,
    SHA2(CONCAT(COALESCE(oon_element.billing_code, ''), '|', COALESCE(oon_element.billing_code_type, '')), 256) as billing_code_id,
    SHA2(CONCAT(COALESCE(tin_info.type, ''), '|', COALESCE(tin_info.value, '')), 256) as tin_id,
    SHA2(CAST(npi_value AS STRING), 256) as provider_id,
    payment_element,
    provider_element,
    rcrd_timestamp,
    ingest_time
  FROM STREAM(source_healthcare_data)
  LATERAL VIEW EXPLODE(out_of_network) as oon
  LATERAL VIEW EXPLODE(array(oon.element)) as oon_element
  LATERAL VIEW EXPLODE(oon_element.allowed_amounts) as allowed_amt
  LATERAL VIEW EXPLODE(array(allowed_amt.element.tin)) as tin_info
  LATERAL VIEW EXPLODE(allowed_amt.element.payments) as payment
  LATERAL VIEW EXPLODE(array(payment.element)) as payment_element
  LATERAL VIEW EXPLODE(payment_element.providers) as provider
  LATERAL VIEW EXPLODE(array(provider.element)) as provider_element
  LATERAL VIEW EXPLODE(provider_element.npi) as npi_value
) exploded_data;

-- SOURCE TABLE DEFINITION
CREATE OR REFRESH STREAMING TABLE source_healthcare_data
COMMENT "Raw healthcare pricing transparency data"
AS SELECT * FROM STREAM READ_FILES(
  '${source_table_path}',
  format => 'delta'
);
